In [1]:
%load_ext autoreload
%autoreload 2
# when debugging the python scripts, with the autoreload the jupyter notebook sync's to the most up to date scripts. See: https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scanpy as sc
import random
from dataprep import *

/media/gambino/students_workdir/nynke/miniconda3/envs/blurry/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
seed = 101
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
rng = np.random.RandomState(seed)

## Loading RF Models & Datasets

In [3]:
#adata_all_spots = sc.read_h5ad("/media/gambino/students_workdir/nynke/data/all_spots_simulated_embeddings_adata.h5ad")
adata_split_cellID = sc.read_h5ad("/media/gambino/students_workdir/nynke/new_model_with_cell_id_left_out_custom_nynke_panel_simulated_embeddings_adata.h5ad")
#adata_mixed_cellID = sc.read_h5ad("/media/gambino/students_workdir/nynke/data/custom_nynke_panel_simulated_embeddings_adata.h5ad")

In [1]:
def map_interval(interval):
    if interval == '0-10':
        return '0-10'
    elif interval in ['10-20', '20-30']:
        return '10-30'
    elif interval in ['30-40', '40-50', '50-60']:
        return '30-60'
    elif interval in ['70-80', '80-90', '90-100']:
        return '70-100'
    elif interval == '100+':
        return '100+'
    else:
        return None
    
def initialize_adata(adata):
    choices = ['strong', 'intermediate', 'low']
    conditions = [
        (adata.obs['prop'] == 0.9) | ((adata.obs['prop'] == 0.4) & (adata.obs['pattern'] == 'protrusion')),
        (adata.obs['prop'] == 0.5) | ((adata.obs['prop'] == 0.2) & (adata.obs['pattern'] == 'protrusion')),
        (adata.obs['prop'] == 0.1) | ((adata.obs['prop'] == 0.0) & (adata.obs['pattern'] == 'protrusion'))
    ]
    adata.obs['pattern_strength'] = np.select(conditions, choices, default='unknown')

    # Include modified RNA count intervals in the adata object
    adata.obs['rna_count'] = adata.obs['n_spots_interval'].apply(map_interval)


    return adata

In [ ]:
def map_interval(interval):
    if interval == '0-10':
        return '0-10'
    elif interval in ['10-20', '20-30']:
        return '10-30'
    elif interval in ['30-40', '40-50', '50-60']:
        return '30-60'
    elif interval in ['70-80', '80-90', '90-100']:
        return '70-100'
    elif interval == '100+':
        return '100+'
    else:
        return None

adata_test.obs['rna_count'] = adata_test.obs['n_spots_interval'].apply(map_interval)

In [5]:
# Creating the simulated genes (1 gene ~300 cells). Method can be found in dataprep.py
# In this case, the genes contain all the counts, iso only low counts. 
pericellular_dict_split_cellID = subsetGenes(adata_split_cellID, 'pericellular', pattern_strength = "strong", mixed_counts = True)
pericellular_dict_split_testID = subsetGenes(adata_split_cellID[adata_split_cellID.obs['cell_id'].isin(adata_split_cellID.uns['test_cellIDs'])], 'pericellular', pattern_strength = "strong", mixed_counts = True)
random_dict_mixed = subsetGenes(adata, 'random', mixed_counts=True)

# Estimate Probability Density Functions with KDE

Use KDEpy, is a quicker to run than KDE implementations of scipy, sklearn and statsmodel, and you can use a bandwidth estimator called ISJ, which is useful for very non-normal data and potentially multimodal data. 

In [13]:
from statsmodels.nonparametric.kernel_density import KDEMultivariate, EstimatorSettings

In [16]:
pericellular_dict_split_cellID[0]

AnnData object with n_obs × n_vars = 317 × 15
    obs: 'pattern', 'random_or_pattern', 'n_spots', 'n_spots_interval', 'cell_id', 'genes', 'rotation', 'rotation_interval', 'blur', 'prop', 'prop_interval', 'corresponding_dapis', 'train_or_val', 'original_image_paths', 'pattern_strength'
    uns: 'test_cellIDs', 'train_cellIDs'
    obsm: 'latent'

In [6]:
settings = EstimatorSettings(efficient=True, n_jobs=15)

use cross validation least squares to select the bandwidth for each dimension. Choose LS over ML because ML makes assumptions about normality etc, whereas LS does not. 

In [17]:
randomgene = KDEMultivariate(data=random_dict_mixed[0].obsm['latent'], var_type='ccccccccccccccc',bw='cv_ls', defaults=settings)

In [18]:
perigene = KDEMultivariate(data=pericellular_dict_split_cellID[0].obsm['latent'], var_type='ccccccccccccccc',bw='cv_ls', defaults=settings)

In [26]:
randomgene

KDE instance
Number of variables: k_vars = 15
Number of samples:   nobs = 317
Variable types:      ccccccccccccccc
BW selection method: cv_ls

Creating the grid to turn the KDE into PDF

In [24]:
# Assuming point_cloud_1 and point_cloud_2 are your two point clouds
combined = np.concatenate([random_dict_mixed[0].obsm['latent'], pericellular_dict_split_cellID[0].obsm['latent']], axis=0)  # Combine the point clouds

# Find the minimum and maximum values across all dimensions and points
min_val = min(np.min(random_dict_mixed[0].obsm['latent']),np.min(pericellular_dict_split_cellID[0].obsm['latent']))
max_val = max(np.max(random_dict_mixed[0].obsm['latent']),np.max(pericellular_dict_split_cellID[0].obsm['latent']))
num_points = 100  # number of points in each latent dimension

# Create the grid
grid = np.linspace(min_val, max_val, num_points)

In [23]:
combined.shape

(634, 15)

In [ ]:
pdf_random = randomgene.pdf(data_predict=grid)
pdf_peri = perigene.pdf(data_predict=grid)

# Chamfer Distance

In [7]:
from scipy.spatial.distance import cdist

In [16]:
def chamfer_L1_distance(point_cloud_1, point_cloud_2):

    # Compute all pairwise Manhattan distances, output matrix = [distance_cloudA1, distance_cloud2]
    distances_1_to_2 = cdist(point_cloud_1, point_cloud_2, metric='cityblock')
    distances_2_to_1 = cdist(point_cloud_2, point_cloud_1, metric='cityblock')

    # Get nearest neighbor for each point from point cloud 1 in point cloud 2 and vice versa
    distances_1_to_2 = np.min(distances_1_to_2, axis=1)
    distances_2_to_1 = np.min(distances_2_to_1, axis=1)

    # Compute the Chamfer distance
    return np.mean(distances_1_to_2) + np.mean(distances_2_to_1)

In [17]:
print(chamfer_L1_distance(random_dict_mixed[0].obsm['latent'], pericellular_dict_split_cellID[0].obsm['latent']))

105.34175945445895


In [21]:
print(chamfer_L1_distance(random_dict_mixed[0].obsm['latent'], random_dict_mixed[2].obsm['latent']))

34.27557134989247
